In [1]:
import torch

In [2]:
# GPU 할당 변경하기
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

# Additional Infos
if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')

Current cuda device  2
GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
from load_data import *

In [4]:
Xtr, Xte, ytr, yte, Ztr, Zte = load_bank_data()
ytr = ytr.astype('float32').reshape((-1,1))
yte = yte.astype('float32').reshape((-1,1))

/home/srp/.pyenv/versions/pytorch15/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


[array([0., 0., 0., ..., 1., 0., 1.]), array([0., 0., 0., ..., 0., 1., 0.]), array([1., 1., 1., ..., 0., 0., 0.]), [], array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([

ValueError: setting an array element with a sequence.

In [ ]:
print('# training samples:', Xtr.shape)
print('# test samples:', Xte.shape)
print(ytr[:10].flatten())
print(Ztr[:10].shape)

In [31]:
N1 = 200
Xtr1 = Xtr[:N1]
ytr1 = ytr[:N1]
Ztr1 = Ztr[:N1]
Xtr2 = Xtr[N1:]
ytr2 = ytr[N1:]
Ztr2 = Ztr[N1:]

In [32]:
from load_data import *

In [33]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [38]:
class Classifier(nn.Module):

    def __init__(self, n_features, n_hidden=32, p_dropout=0.2):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, n_hidden),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Dropout(p_dropout),
            nn.Linear(n_hidden, 1),
        )

    def forward(self, x):
        return torch.sigmoid(self.network(x))

In [39]:
n_features = Xtr.shape[1]
clf = Classifier(n_features=n_features)
clf_criterion = nn.BCELoss()
clf_optimizer = optim.Adam(clf.parameters())

In [40]:
train_data = NPsDataSet(Xtr1, ytr1, Ztr1)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, drop_last=True)

In [57]:
N_CLF_EPOCHS = 10

for epoch in range(N_CLF_EPOCHS):
    for x, y, _ in train_loader:
        clf.zero_grad()
        p_y = clf(x)
        loss = clf_criterion(p_y, y)
        loss.backward()
        clf_optimizer.step()
        print(loss)

tensor(0.4735, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4044, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4848, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3667, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3221, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3960, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4185, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3981, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2870, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3954, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4136, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3466, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3968, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4410, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2893, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3676, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4293, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4764, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3005, grad_fn=<BinaryCrossEntropyBack

tensor(0.3161, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4034, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2275, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2824, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2912, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3836, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2447, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3061, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2634, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2180, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3798, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3603, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2382, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2322, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2880, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3560, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3375, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2356, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3598, grad_fn=<BinaryCrossEntropyBack

tensor(0.2899, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2098, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1213, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2308, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2689, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2849, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1269, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2031, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1647, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1511, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3512, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2070, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1522, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2923, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2733, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2155, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1760, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1616, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3838, grad_fn=<BinaryCrossEntropyBack

tensor(0.1268, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2012, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2452, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1313, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1365, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1204, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2321, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1158, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2486, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1602, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2047, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1906, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1581, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1938, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2554, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2079, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1218, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2264, grad_fn=<BinaryCrossEntropyBack

tensor(0.1089, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0937, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1025, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0795, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0818, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2290, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2591, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0510, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0849, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1536, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2191, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1084, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1163, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1740, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1123, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1456, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1642, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1774, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1553, grad_fn=<BinaryCrossEntropyBack

tensor(0.1134, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0745, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0843, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1297, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1772, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0954, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1999, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1143, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1067, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1271, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0544, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0629, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1262, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0578, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0942, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2192, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1156, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0856, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1643, grad_fn=<BinaryCrossEntropyBack

tensor(0.0452, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0370, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1381, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1315, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0732, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0509, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1302, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0492, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0702, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0933, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2506, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1655, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0907, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0760, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1459, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0669, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0851, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1875, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0999, grad_fn=<BinaryCrossEntropyBack

In [24]:
class Adversary(nn.Module):

    def __init__(self, n_sensitive, n_hidden=32):
        super(Adversary, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(1, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_sensitive),
        )

    def forward(self, x):
        return F.sigmoid(self.network(x))

In [63]:
# lambdas = torch.Tensor([200, 30])
lambdas = torch.Tensor([1])
adv = Adversary(Ztr1.shape[1])
adv_criterion = nn.BCELoss(reduce=False)
adv_optimizer = optim.Adam(adv.parameters())

/home/srp/.pyenv/versions/pytorch15/lib/python3.8/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [64]:
N_ADV_EPOCHS = 5

In [66]:
print(lambdas)
for epoch in range(N_ADV_EPOCHS):
    for x, _, z in train_loader:
        adv.zero_grad()
        p_y = clf(x).detach()
        p_z = adv(p_y)
#         print(lambdas.shape)
#         print((adv_criterion(p_z, z)).shape)
        loss = (adv_criterion(p_z, z) * lambdas).mean()
        loss.backward()
        adv_optimizer.step()

tensor([1.])


In [67]:
N_EPOCH_COMBINED = 165

for epoch in range(1, N_EPOCH_COMBINED):

    # Train adversary
    for x, y, z in train_loader:
        adv.zero_grad()
        p_y = clf(x)
        p_z = adv(p_y)
        loss_adv = (adv_criterion(p_z, z) * lambdas).mean()
        loss_adv.backward()
        adv_optimizer.step()

    # Train classifier on single batch
    for x, y, z in train_loader:
        pass  # Ugly way to get a single batch
    clf.zero_grad()
    p_y = clf(x)
    p_z = adv(p_y)
    loss_adv = (adv_criterion(p_z, z) * lambdas).mean()
    clf_loss = clf_criterion(p_y, y) - (adv_criterion(adv(p_y), z) * lambdas).mean()
    clf_loss.backward()
    clf_optimizer.step()

/home/srp/.pyenv/versions/pytorch15/lib/python3.8/site-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


In [15]:
from fair_eval import calculate_prule_clf, calculate_odds_clf, calculate_parity_reg, calculate_group_loss,l2_loss, calculate_overall_accuracy

In [58]:
pred = (clf(torch.tensor(Xte).float())>0.5).float()
pred = pred.cpu().detach().numpy().flatten()

In [59]:
pred_ = clf(torch.tensor(Xte).float())
pred_ = pred_.cpu().detach().numpy().flatten()

In [60]:
print(pred_)

[2.3538221e-01 1.2413255e-03 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0000000e+00 1.0000000e+00 2.0511901e-01 1.0000000e+00 9.6937490e-01
 9.9749798e-01 9.9981707e-01 1.0000000e+00 9.9999428e-01 9.9767429e-01
 1.0000000e+00 9.9999225e-01 8.9258230e-01 1.0000000e+00 1.5512447e-01
 3.9171115e-02 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 9.9999976e-01 1.0000000e+00 1.0000000e+00 1.0000000e+00 6.2504935e-04
 1.0000000e+00 1.0000000e+00 9.9999893e-01 9.9998915e-01 1.0654778e-04
 8.3752722e-01 1.5092101e-03 6.6149596e-04 9.9966538e-01 1.0000000e+00
 1.0000000e+00 9.9997866e-01 1.5397543e-02 1.0000000e+00 9.9881738e-01
 9.7381896e-01 9.1319686e-01 9.9588531e-01 1.0000000e+00 9.7905046e-01
 5.4246902e-02 1.6641982e-04 1.0000000e+00 1.0000000e+00 1.7419867e-03
 1.0000000e+00 1.0000000e+00 1.3249847e-01 7.8252349e-03 4.2536462e-04
 9.9997997e-01 4.9439285e-02 1.0000000e+00 5.3632051e-01 1.9662890e-03
 1.0000000e+00 9.9866188e-01 9.9999988e-01 9.9997520e-01 2.8819951e-01
 9.997

In [62]:
calculate_prule_clf(pred,yte,Zte)

disparate impact:  78.72340425531915
disparate misclassification rate:  55.769230769230774
disparate false positive rate: 89.1891891891892
disparate false negative rate: 44.94252873563219


In [64]:
calculate_odds_clf(pred,yte.flatten(),Zte)

equalized opportunity for 0.0 : 89.1891891891892
equalized opportunity for 1.0 : 81.34008570315544


In [46]:
calculate_group_loss(bce_loss,pred_,y_te,xs_te)

NameError: name 'bce_loss' is not defined

In [61]:
calculate_overall_accuracy(pred,yte.flatten())

0.7233333333333334

In [78]:
min([1,3,5,6])

1